In [1]:
# activiate inline plotting
%matplotlib inline

import sys
sys.path.append("/home/kaspar")

import numpy as np
import scipy as sp
import pandas as pd

from y10k_prediction_methods.helper_functions import summarise_Rsq

### Get data

In [2]:
from y10k_prediction_methods.data_import import get_data_with_parents

file_name = 'data/y10k_hybrids_Yield.hdf5'
Y, snps, K, parent1, parent2, individuals, dataset, environments = get_data_with_parents(file_name)

/home/kaspar/anaconda/lib/python2.7/site-packages/limix/io/genotype_reader.py:248: DeprecationWarning: openFile() is pending deprecation, use open_file() instead. You may use the pt2to3 tool to update your source code.
  self.f = tables.openFile(self.file_name,'r')
/home/kaspar/anaconda/lib/python2.7/site-packages/limix/io/phenotype_reader.py:31: DeprecationWarning: openFile() is pending deprecation, use open_file() instead. You may use the pt2to3 tool to update your source code.
  self.f = tables.openFile(self.file_name,'r')


### 4-fold CV partitions into test and two sets of training sets (distant and close relatives)

In [3]:
from y10k_prediction_methods.train_and_test_sets import get_4foldCV_close_and_distant

sp.random.seed(0)
Itest_list, Idistant_list, Iclose_list = get_4foldCV_close_and_distant(parent1, parent2)

### BLUP confidence (i.e. SD of predictive distribution)

In [4]:
from IPython.parallel import Client
c = Client()
cluster = c[:]

In [5]:
myfunction_blup = lambda j: get_BLUPs_with_confidence(Y[:, j:j+1], K, Itrain, Itest)

cluster.execute('''
import sys
sys.path.append("/home/kaspar/yeast/code/")
from y10k_prediction_methods.BLUP import get_BLUPs_with_confidence
''')

sd = np.sqrt(Y.var(axis=0))
Ynorm = (Y - Y.mean(axis=0)) / sd

In [6]:
Itest = Itest_list[0]
Itrain = Idistant_list[0]
mydict=dict(Y=Ynorm, K=K, Itrain=Itrain, Itest=Itest)
cluster.push(mydict)
res = cluster.map_sync(myfunction_blup, range(Y.shape[1]))

predictive_sd1_all = np.array([obj["predictive_sd"] for obj in res]).T
predictive_sd1 = predictive_sd1_all.mean(axis=0)
pred1 = np.array([obj["pred"].ravel() for obj in res]).T
resid1 = Ynorm[Itest, :] - pred1
resid_sd1 = np.sqrt(resid1.var(axis=0))

In [7]:
Itest = Itest_list[0]
Itrain = Iclose_list[0]
mydict=dict(Y=Ynorm, K=K, Itrain=Itrain, Itest=Itest)
cluster.push(mydict)
res = cluster.map_sync(myfunction_blup, range(Y.shape[1]))

predictive_sd2_all = np.array([obj["predictive_sd"] for obj in res]).T
predictive_sd2 = predictive_sd2_all.mean(axis=0)
pred2 = np.array([obj["pred"].ravel() for obj in res]).T
resid2 = Ynorm[Itest, :] - pred2
resid_sd2 = np.sqrt(resid2.var(axis=0))

In [8]:
temp = np.row_stack((predictive_sd1, resid_sd1, predictive_sd2, resid_sd2))
df = pd.DataFrame(temp, columns=environments)
df["SD_type"] = ["predictive_SD", "residuals_SD"]*2
df["relatives"] = ["distant", "distant", "close", "close"]
df.to_csv("output/close_distant_BLUP_confidence.csv")

### QTL confidence (i.e. SD of predictive distribution)

In [16]:
from IPython.parallel import Client
c = Client()
cluster = c[:]

In [17]:
maxiter = 25
pred_nQTLs = range(maxiter+1)
myfunction_qtl_additive = lambda j: QTL_iid_predictions(Y, j, snps, Itrain, Itest, pred_nQTLs, maxiter=maxiter)

cluster.execute('''
import sys
sys.path.append("/home/kaspar")
from y10k_prediction_methods.QTL_fitting import QTL_iid_predictions
''')

sd = np.sqrt(Y.var(axis=0))
Ynorm = (Y - Y.mean(axis=0)) / sd

In [18]:
Itest = Itest_list[0]
Itrain = Iclose_list[0]
mydict=dict(Y=Ynorm, snps=snps, Itrain=Itrain, Itest=Itest, pred_nQTLs=pred_nQTLs, maxiter=maxiter)
cluster.push(mydict)
res_close = cluster.map_sync(myfunction_qtl_additive, range(Y.shape[1]))

Itest = Itest_list[0]
Itrain = Idistant_list[0]
mydict=dict(Y=Ynorm, snps=snps, Itrain=Itrain, Itest=Itest, pred_nQTLs=pred_nQTLs, maxiter=maxiter)
cluster.push(mydict)
res_distant = cluster.map_sync(myfunction_qtl_additive, range(Y.shape[1]))

In [19]:
from y10k_prediction_methods.QTL_fitting import get_predictions_iid_with_weights

P = Y.shape[1]
predictive_s2_close = sp.zeros(P)
predictive_s2_distant = sp.zeros(P)
pred_error_close = sp.zeros(P)
pred_error_distant = sp.zeros(P)
for j in range(P):
    covs = res_close[j]["covs"]
    pred_error_close[j] = (res_close[j]["pred"] - Ynorm[Itest, j]).var(axis=0)
    _, _, predictive_s2_close[j] = get_predictions_iid_with_weights(Ynorm[:, j], covs, Itrain, Itest)
    covs = res_distant[j]["covs"]
    pred_error_distant[j] = (res_distant[j]["pred"] - Ynorm[Itest, j]).var(axis=0)
    _, _, predictive_s2_distant[j] = get_predictions_iid_with_weights(Ynorm[:, j], covs, Itrain, Itest)

In [20]:
df = pd.DataFrame(np.sqrt(np.row_stack((predictive_s2_distant, pred_error_distant, predictive_s2_close, pred_error_close))), columns=environments)
df["SD_type"] = ["predictive_SD", "residuals_SD"]*2
df["relatives"] = ["distant", "distant", "close", "close"]
df.to_csv("output/close_distant_QTL_confidence.csv")